In [ ]:
########################################
# 1. Import Packages
########################################
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy.stats import t as t_dist

########################################
# 2. Read the CSV File
########################################
file_path = r"path to your DB"
df = pd.read_csv(file_path)

print(df.head())

########################################
# 3. Ensure 'sex' is categorical, check columns
########################################
df['sex'] = df['sex'].astype('category')
print("\nColumns in DataFrame:", df.columns.tolist())

########################################
# 4. Define Outcome & Phenotypes
########################################
outcome = "ca_delta"

phenotypes = [
    "Visceral_adipose_tissue_volume",
    "Subcutaneous_fat_body",
    "Trunk_fat_mass",
    "Whole_body_fat_mass",
    "Muscle_fat_infiltration",
    "Android_fat_mass",
    "Gynoid_fat_mass"
]

# Example covariates (adjust as needed)
covariates = "age_at_mri"

########################################
# 5. Fit Models & Extract Both Fem & Male
########################################
results_list = []

for pheno in phenotypes:
    formula = f"{outcome} ~ sex * {pheno} + {covariates}"
    print(f"\nFitting model: {formula}")
    model = smf.ols(formula=formula, data=df).fit()
    
    # Degrees of freedom for t-distribution (for male slope test)
    df_resid = model.df_resid

    # 1) Female main effect (Beta_Fem) => the phenotype name itself, e.g. "Trunk_fat_mass"
    beta_fem_name = pheno
    beta_fem = model.params.get(beta_fem_name, np.nan)
    se_fem   = model.bse.get(beta_fem_name, np.nan)
    t_fem    = model.tvalues.get(beta_fem_name, np.nan)
    p_fem    = model.pvalues.get(beta_fem_name, np.nan)
    
    # 2) Interaction (M-F difference) => usually "sex[T.Male]:Phenotype"
    interaction_term = None
    interaction_candidates = [
        c for c in model.params.index if (":" in c and pheno in c)
    ]
    if len(interaction_candidates) > 0:
        interaction_term = interaction_candidates[0]
        diff_mf = model.params.get(interaction_term, np.nan)
        diff_mf_se = model.bse.get(interaction_term, np.nan)
        diff_mf_t  = model.tvalues.get(interaction_term, np.nan)
        diff_mf_p  = model.pvalues.get(interaction_term, np.nan)
    else:
        diff_mf = np.nan
        diff_mf_se = np.nan
        diff_mf_t  = np.nan
        diff_mf_p  = np.nan
    
    # 3) Beta for Males = Beta_Fem + Interaction
    beta_male = beta_fem + diff_mf if not (pd.isna(beta_fem) or pd.isna(diff_mf)) else np.nan
    
    # 4) Compute male variance via covariance matrix
    cov_params = model.cov_params()
    if (beta_fem_name in cov_params.columns) and (interaction_term in cov_params.columns) and (interaction_term is not None):
        var_male = (cov_params.loc[beta_fem_name, beta_fem_name]
                    + cov_params.loc[interaction_term, interaction_term]
                    + 2 * cov_params.loc[beta_fem_name, interaction_term])
    else:
        var_male = np.nan
    
    se_male = np.sqrt(var_male) if not np.isnan(var_male) else np.nan
    
    # 5) t-value & p-value for Beta_Male
    if not np.isnan(se_male) and (se_male != 0):
        t_male = beta_male / se_male
        # two-sided p-value using t distribution
        p_male = 2.0 * (1.0 - t_dist.cdf(abs(t_male), df_resid))
    else:
        t_male = np.nan
        p_male = np.nan

    # Compile row data
    row_data = {
        "Phenotype"      : pheno,

        "Beta_Fem"       : beta_fem,
        "SE_Fem"         : se_fem,
        "t_Fem"          : t_fem,
        "p_Fem"          : p_fem,

        "Beta_Male"      : beta_male,
        "SE_Male"        : se_male,
        "t_Male"         : t_male,
        "p_Male"         : p_male,

        "Diff(M-F)"      : diff_mf,
        "SE(Diff)"       : diff_mf_se,
        "t(Diff)"        : diff_mf_t,
        "p(Diff)"        : diff_mf_p
    }
    results_list.append(row_data)

########################################
# 6. Combine into a DataFrame
########################################
final_results = pd.DataFrame(results_list)
print("\n=============================")
print("Female & Male Results")
print("=============================")
print(final_results)

########################################
# 7. (Optional) P-value Rounding / Capping
########################################
def format_p_value(p):
    # If missing or not numeric
    if pd.isna(p):
        return p
    # If numeric, cap below 0.0001
    if p < 0.0001:
        return "<0.0001"
    return f"{p:.4f}"  # 4 decimal places

for col in ["p_Fem", "p_Male", "p(Diff)"]:
    final_results[col] = final_results[col].apply(format_p_value)

print("\n=============================")
print("After P-value formatting")
print("=============================")
print(final_results)